In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time

In [6]:
url= "https://learningsuite.byu.edu/.w60q/student/top/prioritizer"
r = requests.get(url)
text = r.text
soup= BeautifulSoup(r.text)


In [17]:
#trs = soup.find_all('tr', {'class':'h-10 bg-base'}{'data':'h-10 bg-base'})
trs = soup.find_all("tr", class_="h-10 bg-base")
trs


[]